In [43]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="")
index = pc.Index("breakout-ai-index")
import os
from langchain_openai import OpenAIEmbeddings

os.environ['OPENAI_API_KEY'] = ''
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.5},
)

In [44]:
from langchain.tools.retriever import create_retriever_tool

In [45]:
backtraderSearch = create_retriever_tool(
    retriever,
    "backtrader_search",
    "This tool contains the Documentation for the backtrader library which we are going to be using, when you are given a strategy, use this tool to see what indicators you are going to use, and how will you make a class using those indicators.",
)

In [46]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import ChatOpenAI, OpenAI
tools = [backtraderSearch]

In [71]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [72]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You will always be given a trading strategy as an input, the data will be provided to you, so you only hae to take help of the backtrader_search tool to see how will you make the class which I will later copy and paste and run it using backtrader api. Keep sure you are only making the class and nothing else. Also make the class such that when I run the cerebro engine I see what is hapening every day, that is after you are done with capturing the logic, make logs of the trade every day, example: class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        if self.dataclose[0] < self.dataclose[-1]:
            # current close less than previous close

            if self.dataclose[-1] < self.dataclose[-2]:
                # previous close less than the previous close

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.buy()
                
                see here in this very basic stategy we can see that how the buy/sell is printed, you have to do this for every class you make. Also make sure your position is always there, that is if you are at a buy position you can only sell, if you are at a sell position you can only buy next. Also make sure that array don't go out of range and also the function data.close is used properly.""",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [73]:
llm_with_tools = model.bind_tools(tools)

In [74]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [75]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [76]:
# input
# input thing



In [77]:
ans = list(agent_executor.stream({"input": """whenever the sma50 crosses sma200 we buy else we sell"""}))



> Entering new AgentExecutor chain...

Invoking: `backtrader_search` with `{'query': 'SMA'}`


16/08/24, 11:03 PM
Indicators - Reference - Backtrader
Page 140 of 187
https://www.backtrader.com/docu/indautoref/
plotlinevalues (True)
plotvaluetags (True)
plotymargin (0.0)
plotyhlines ([])
plotyticks ([])
plothlines ([])
plotforce (False)
PlotLines:
signal:
SmoothedMovingAverage
Alias:
SMMA, WilderMA, MovingAverageSmoothed, MovingAverageWilder, ModiSedMovingAverage
Smoothing Moving Average used by Wilder in his 1978 book New Concepts in Technical Trading
DeSned in his book originally as:
new_value = (old_value * (period - 1) + new_data) / period
Can be expressed as a SmoothingMovingAverage with the following factors:
self.smfactor -> 1.0 / period
self.smfactor1 -> 1.0 - self.smfactor
Formula:
movav = prev * (1.0 - smoothfactor) + newdata * smoothfactor
See also:
http://en.wikipedia.org/wiki/Moving_average#ModiSed_moving_average
Lines:
smma
Params:
Invoking: `backtrader_search` with `{'q

In [79]:
print(ans[-1]['output'])

```python
import backtrader as bt

class SMA_Cross_Strategy(bt.Strategy):
    # Define the parameters for the moving averages
    params = (
        ('sma_short', 50),
        ('sma_long', 200),
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Add the moving averages
        self.sma_short = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.sma_short)
        self.sma_long = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.sma_long)
        self.cross_over = bt.indicators.CrossOver(self.sma_short, self.sma_long)

        # To keep track of pending orders
        self.order = None

    def next(self):
        # Log the current closing price and the SMAs
        self.log('Close, %.2f, SMA50, %.2f, SMA200, %.2f' % (self.data.close[0], self.sma_short[0], self.sma_long[0]))

    

In [55]:
print(ans)

[{'actions': [ToolAgentAction(tool='backtrader_search', tool_input={'query': 'SMA'}, log="\nInvoking: `backtrader_search` with `{'query': 'SMA'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_T3MApY2Z99TN7xLtDh6ztVa1', 'function': {'arguments': '{"query": "SMA"}', 'name': 'backtrader_search'}, 'type': 'function'}, {'index': 1, 'id': 'call_ViGaDTEzCi4NH3gH64Reecg1', 'function': {'arguments': '{"query": "CrossOver"}', 'name': 'backtrader_search'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_25624ae3a5'}, id='run-c7a6ef1e-5eb7-456a-bc8f-38313f75fd8e', tool_calls=[{'name': 'backtrader_search', 'args': {'query': 'SMA'}, 'id': 'call_T3MApY2Z99TN7xLtDh6ztVa1', 'type': 'tool_call'}, {'name': 'backtrader_search', 'args': {'query': 'CrossOver'}, 'id': 'call_ViGaDTEzCi4NH3gH64Reecg1', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'backtrader_se

In [56]:
import yfinance as yf
import pandas as pd

symbol = "RELIANCE.NS"
interval = "1d"
start_date = "2020-08-11"
end_date = "2024-08-17"

df = yf.download(tickers=symbol, interval=interval, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [57]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-08-11,1970.419678,1993.679321,1938.622314,1969.496704,1942.791626,23414153
2020-08-12,1957.682251,1979.834351,1944.575684,1963.774048,1937.146606,15132308
2020-08-13,1964.143311,1974.203979,1947.529297,1958.651489,1932.093384,12415161
2020-08-14,1959.112915,1990.910278,1928.377075,1951.036743,1924.582031,16612317
2020-08-17,1942.914307,1942.914307,1910.701660,1930.315430,1904.141602,15489533
...,...,...,...,...,...,...
2024-08-09,2920.000000,2953.000000,2912.000000,2948.600098,2938.626465,3124888
2024-08-12,2940.000000,2946.000000,2915.500000,2921.250000,2911.368896,4092292
2024-08-13,2921.500000,2940.149902,2916.000000,2927.250000,2917.348633,3132532


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 994 entries, 2020-08-11 to 2024-08-16
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       994 non-null    float64
 1   High       994 non-null    float64
 2   Low        994 non-null    float64
 3   Close      994 non-null    float64
 4   Adj Close  994 non-null    float64
 5   Volume     994 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 54.4 KB


In [59]:
import backtrader as bt
cerebro = bt.Cerebro()


In [60]:
data_feed = bt.feeds.PandasData(
    dataname=df)

cerebro.adddata(data_feed)

In [103]:
class SMA_Cross_Strategy(bt.Strategy):
    # Define the parameters for the moving averages
    params = (
        ('sma_short', 50),
        ('sma_long', 200),
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Add the moving averages
        self.sma_short = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.sma_short)
        self.sma_long = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.sma_long)
        self.cross_over = bt.indicators.CrossOver(self.sma_short, self.sma_long)

        # To keep track of pending orders
        self.order = None

    def next(self):
        # Log the current closing price and the SMAs
        self.log('Close, %.2f, SMA50, %.2f, SMA200, %.2f' % (self.data.close[0], self.sma_short[0], self.sma_long[0]))

        # Check if an order is pending
        if self.order:
            return  # Awaiting a pending order

        # Check for a crossover
        if self.cross_over > 0:  # SMA50 crosses above SMA200
            # BUY, BUY, BUY!!! (with all possible default parameters)
            self.log('BUY CREATE, %.2f' % self.data.close[0])
            self.order = self.buy()

        elif self.cross_over < 0:  # SMA50 crosses below SMA200
            # SELL, SELL, SELL!!! (with all possible default parameters)
            self.log('SELL CREATE, %.2f' % self.data.close[0])
            self.order = self.sell()


In [ ]:
ans = """class SMA_Cross_Strategy(bt.Strategy):
    # Define the parameters for the moving averages
    params = (
        ('sma_short', 50),
        ('sma_long', 200),
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Add the moving averages
        self.sma_short = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.sma_short)
        self.sma_long = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.sma_long)
        self.cross_over = bt.indicators.CrossOver(self.sma_short, self.sma_long)

        # To keep track of pending orders
        self.order = None

    def next(self):
        # Log the current closing price and the SMAs
        self.log('Close, %.2f, SMA50, %.2f, SMA200, %.2f' % (self.data.close[0], self.sma_short[0], self.sma_long[0]))

        # Check if an order is pending
        if self.order:
            return  # Awaiting a pending order

        # Check for a crossover
        if self.cross_over > 0:  # SMA50 crosses above SMA200
            # BUY, BUY, BUY!!! (with all possible default parameters)
            self.log('BUY CREATE, %.2f' % self.data.close[0])
            self.order = self.buy()

        elif self.cross_over < 0:  # SMA50 crosses below SMA200
            # SELL, SELL, SELL!!! (with all possible default parameters)
            self.log('SELL CREATE, %.2f' % self.data.close[0])
            self.order = self.sell()
"""

In [104]:
cerebro = bt.Cerebro()
cerebro.addstrategy(SMA_Cross_Strategy)
cerebro.adddata(data_feed)
cerebro.broker.setcash(30000)
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 30000.00
2021-06-01, Close, 2001.89, SMA50, 1832.10, SMA200, 1886.52
2021-06-02, Close, 2031.75, SMA50, 1834.79, SMA200, 1886.86
2021-06-03, Close, 2039.51, SMA50, 1838.49, SMA200, 1887.26
2021-06-04, Close, 2021.83, SMA50, 1840.50, SMA200, 1887.62
2021-06-07, Close, 2055.89, SMA50, 1843.54, SMA200, 1888.25
2021-06-08, Close, 2044.08, SMA50, 1845.89, SMA200, 1888.69
2021-06-09, Close, 2007.52, SMA50, 1848.25, SMA200, 1888.89
2021-06-10, Close, 2014.91, SMA50, 1851.78, SMA200, 1889.29
2021-06-11, Close, 2042.74, SMA50, 1855.82, SMA200, 1889.89
2021-06-14, Close, 2072.04, SMA50, 1859.80, SMA200, 1890.58
2021-06-15, Close, 2076.75, SMA50, 1864.35, SMA200, 1891.36
2021-06-16, Close, 2041.31, SMA50, 1867.86, SMA200, 1891.70
2021-06-17, Close, 2045.04, SMA50, 1871.97, SMA200, 1892.18
2021-06-18, Close, 2054.27, SMA50, 1876.43, SMA200, 1892.69
2021-06-21, Close, 2064.98, SMA50, 1880.76, SMA200, 1893.41
2021-06-22, Close, 2054.37, SMA50, 1884.83, SMA200, 1894.05
2021-

In [83]:
print(cerebro.run())

2021-06-01, Close, 2001.89, SMA50, 1832.10, SMA200, 1886.52
2021-06-02, Close, 2031.75, SMA50, 1834.79, SMA200, 1886.86
2021-06-03, Close, 2039.51, SMA50, 1838.49, SMA200, 1887.26
2021-06-04, Close, 2021.83, SMA50, 1840.50, SMA200, 1887.62
2021-06-07, Close, 2055.89, SMA50, 1843.54, SMA200, 1888.25
2021-06-08, Close, 2044.08, SMA50, 1845.89, SMA200, 1888.69
2021-06-09, Close, 2007.52, SMA50, 1848.25, SMA200, 1888.89
2021-06-10, Close, 2014.91, SMA50, 1851.78, SMA200, 1889.29
2021-06-11, Close, 2042.74, SMA50, 1855.82, SMA200, 1889.89
2021-06-14, Close, 2072.04, SMA50, 1859.80, SMA200, 1890.58
2021-06-15, Close, 2076.75, SMA50, 1864.35, SMA200, 1891.36
2021-06-16, Close, 2041.31, SMA50, 1867.86, SMA200, 1891.70
2021-06-17, Close, 2045.04, SMA50, 1871.97, SMA200, 1892.18
2021-06-18, Close, 2054.27, SMA50, 1876.43, SMA200, 1892.69
2021-06-21, Close, 2064.98, SMA50, 1880.76, SMA200, 1893.41
2021-06-22, Close, 2054.37, SMA50, 1884.83, SMA200, 1894.05
2021-06-23, Close, 2035.54, SMA50, 1888.

In [ ]:
fig = cerebro.plot(style='bar', volume=False)[0][0]
fig.set_size_inches(12, 8)
fig.savefig('backtrader_plot.png', dpi=300)


<IPython.core.display.Javascript object>

In [100]:
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 30997.79
2021-06-01, Close, 2001.89, SMA50, 1832.10, SMA200, 1886.52
2021-06-02, Close, 2031.75, SMA50, 1834.79, SMA200, 1886.86
2021-06-03, Close, 2039.51, SMA50, 1838.49, SMA200, 1887.26
2021-06-04, Close, 2021.83, SMA50, 1840.50, SMA200, 1887.62
2021-06-07, Close, 2055.89, SMA50, 1843.54, SMA200, 1888.25
2021-06-08, Close, 2044.08, SMA50, 1845.89, SMA200, 1888.69
2021-06-09, Close, 2007.52, SMA50, 1848.25, SMA200, 1888.89
2021-06-10, Close, 2014.91, SMA50, 1851.78, SMA200, 1889.29
2021-06-11, Close, 2042.74, SMA50, 1855.82, SMA200, 1889.89
2021-06-14, Close, 2072.04, SMA50, 1859.80, SMA200, 1890.58
2021-06-15, Close, 2076.75, SMA50, 1864.35, SMA200, 1891.36
2021-06-16, Close, 2041.31, SMA50, 1867.86, SMA200, 1891.70
2021-06-17, Close, 2045.04, SMA50, 1871.97, SMA200, 1892.18
2021-06-18, Close, 2054.27, SMA50, 1876.43, SMA200, 1892.69
2021-06-21, Close, 2064.98, SMA50, 1880.76, SMA200, 1893.41
2021-06-22, Close, 2054.37, SMA50, 1884.83, SMA200, 1894.05
2021-